In [1]:
import numpy as np
import pandas as pd
import requests
import math

In [18]:
df = pd.read_csv('jfall.csv')
df = df[(df['坐标'] != np.nan)]
df[["套数", "单价"]] = df[["套数", "单价"]].fillna(-1)
df[["户型", "类别"]] = df[["户型", "类别"]].fillna(-1)
df[["入围积分"]] = df[["入围积分"]].fillna("未开盘")

In [19]:
df

,区域,板块,项目名称,套数,开盘时间,单价,套均总价,户型,类别,社保系数,是否触发积分,入围积分,标准化积分,坐标
0,嘉定,江桥,保利云上澄光,232,6/18/2021,56000,NaN,-1,-1,0.11,触发,66.28,64.8,"121.310222,31.268965"
1,嘉定,安亭,中骏璟尚,184,6/8/2021,37000,NaN,-1,-1,0.10,触发,60.5,60.5,"121.161699,31.285837"
2,嘉定,江桥,南山虹桥领峯,-1,7/13/2021,57500,NaN,-1,-1,0.11,触发,58.36,57.6,"121.31576,31.267595"
3,嘉定,南翔,融信海纳印象,340,7/20/2021,59000,NaN,-1,-1,0.11,触发,52.09,51.9,"121.307735,31.301541"
4,嘉定,菊园新区,金地峯范,326,5/30/2021,46500,NaN,-1,-1,0.10,触发,51.9,51.9,"121.228507,31.392958"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,奉贤,海湾,大名城映园,432,NaN,22976,NaN,96-124平,洋房、小高层,NaN,NaN,未开盘,NaN,"121.747068,30.872966"
299,奉贤,南桥,中建海樾府,1190,NaN,41000,NaN,89-160平,小高层,NaN,NaN,未开盘,NaN,"121.487426,30.956796"
300,奉贤,海湾,泷悦蓝湾,400,NaN,31862,NaN,95-125平,高层,NaN,NaN,未开盘,NaN,"121.743277,30.87293"
301,奉贤,南桥,悦澜雅苑,待定,NaN,38000,NaN,86-135平米,高层,NaN,NaN,未开盘,NaN,"121.442955,30.965119"


In [20]:
def check_duplicate(json_in_html_file, row):
    for record in json_in_html_file:
        if record["title"] == row["项目名称"] and record["point"] == row["坐标"]:
            return True

    return False

def deviate_coordinate(cord):
    deviate_delta = 0.0005
    split_cord = cord.split(",")
    if len(split_cord) != 2:
        return cord
    else:
        s = str(float(split_cord[0])+deviate_delta) + "," + str(float(split_cord[1]) + deviate_delta)
        print (s)
        print (cord)
        return s


In [22]:
def get_point_type(score):
    if score == "不触发":
        return 1

    if score == "未开盘":
        return 5

    score = float(score)
    if score < 50:
        return 1
    elif score < 60:
        return 2
    elif score < 70:
        return 3
    elif score < 300:
        return 4

    return 5
    

json_in_html_file = []
for index, row in df.iterrows():
    entry = {}
    entry["title"] = row["项目名称"]
    if check_duplicate(json_in_html_file, row):
        entry["point"] = deviate_coordinate(row["坐标"])
    else:
        entry["point"] = str(row["坐标"])
    #print (str(row["坐标"]))

    entry["pointType"] = str(get_point_type((row["入围积分"])))
    entry["score"] = row["入围积分"]

    if str(row["套数"]).isdecimal():
        entry["poolsize"] = str(int(row["套数"]))
    else:
        entry["poolsize"] = row["套数"]

    if str(row["单价"]).isdecimal():
        entry["unitPrice"] =  str(int(row["单价"]))
    else:
        entry["unitPrice"] = row["单价"]

    entry["type"] = row["类别"]
    entry["roomtype"] = row["户型"]
    entry["url"] = ""

    if (entry["point"]) != 'nan':
        json_in_html_file.append(entry)

['121.228507', '31.392958']
121.229007,31.393458
['121.307735', '31.301541']
121.308235,31.302041
['121.436379', '30.95948']
121.436879,30.959979999999998
['121.51607', '30.92473']
121.51657,30.92523
['121.436379', '30.95948']
121.436879,30.959979999999998
['121.732595', '30.933689']
121.733095,30.934189
['121.662706', '30.909721']
121.663206,30.910221
['121.475091', '30.908286']
121.47559100000001,30.908786
['121.662706', '30.909721']
121.663206,30.910221
['121.405326', '31.31953']
121.405826,31.32003
['121.429076', '31.391743']
121.429576,31.392243
['121.400086', '31.358421']
121.400586,31.358921
['121.343594', '31.393956']
121.344094,31.394455999999998
['121.403842', '31.300773']
121.404342,31.301273
['121.523705', '31.270599']
121.52420500000001,31.271099
['121.510597', '31.321788']
121.511097,31.322288
['121.225308', '31.004858']
121.225808,31.005357999999998
['121.175914', '31.085558']
121.17641400000001,31.086057999999998
['121.297069', '31.084758']
121.297569,31.085258
['121.18

In [23]:
df.head(3)

,区域,板块,项目名称,套数,开盘时间,单价,套均总价,户型,类别,社保系数,是否触发积分,入围积分,标准化积分,坐标
0,嘉定,江桥,保利云上澄光,232,6/18/2021,56000,NaN,-1,-1,0.11,触发,66.28,64.8,"121.310222,31.268965"
1,嘉定,安亭,中骏璟尚,184,6/8/2021,37000,NaN,-1,-1,0.10,触发,60.5,60.5,"121.161699,31.285837"
2,嘉定,江桥,南山虹桥领峯,-1,7/13/2021,57500,NaN,-1,-1,0.11,触发,58.36,57.6,"121.31576,31.267595"


In [24]:
json_in_html_file

[{'title': '保利云上澄光',
  'point': '121.310222,31.268965',
  'pointType': '3',
  'score': 66.28,
  'poolsize': '232',
  'unitPrice': '56000',
  'type': -1,
  'roomtype': -1,
  'url': ''},
 {'title': '中骏璟尚',
  'point': '121.161699,31.285837',
  'pointType': '3',
  'score': 60.5,
  'poolsize': '184',
  'unitPrice': '37000',
  'type': -1,
  'roomtype': -1,
  'url': ''},
 {'title': '南山虹桥领峯',
  'point': '121.31576,31.267595',
  'pointType': '2',
  'score': 58.36,
  'poolsize': -1,
  'unitPrice': '57500',
  'type': -1,
  'roomtype': -1,
  'url': ''},
 {'title': '融信海纳印象',
  'point': '121.307735,31.301541',
  'pointType': '2',
  'score': 52.09,
  'poolsize': '340',
  'unitPrice': '59000',
  'type': -1,
  'roomtype': -1,
  'url': ''},
 {'title': '金地峯范',
  'point': '121.228507,31.392958',
  'pointType': '2',
  'score': 51.9,
  'poolsize': '326',
  'unitPrice': '46500',
  'type': -1,
  'roomtype': -1,
  'url': ''},
 {'title': '万科西郊都会',
  'point': '121.195717,31.272131',
  'pointType': '1',
  'score'

In [ ]:
r.json()